In [5]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
git 

ImportError: 
AutoModel requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.


In [4]:
!conda install pytorch torchvision torchaudio -c pytorch
y


Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3/envs/nlp_masterthesis

  added / updated specs:
    - pytorch
    - torchaudio
    - torchvision


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2022.9.24          |  py310hecd8cb5_0         155 KB
    ffmpeg-4.3                 |       h0a44026_0        10.1 MB  pytorch
    gettext-0.21.0             |       h7535e17_0         2.6 MB
    gmp-6.2.1                  |       he9d5cce_3         525 KB
    gnutls-3.6.15              |       hed9c0bf_0         974 KB
    lame-3.100                 |       h1de35cc_0         316 KB
    libidn2-2.3.2              |       h9ed2024_0          85 KB
    libtasn1-4.16.0            |       h9ed2024_0          53 KB
    libunistring-0.9.10        |       h9ed2024_0         519 KB
    nettle-3.7.3               |       h230ac6f_1         380 KB
    openh264

In [ ]:
# github bio bert - https://github.com/dmis-lab/biobert/blob/master/modeling.py